# Trigramdata


For å kjøre notebooken er det bare å aktiver cellene og se på kommentarene. De første to er boilerplate, noe importkode og kode for å ting til å gå, og som gjør at jeg eventuelt kan legget til mer funksjonalitet.

In [1]:
import json
import pandas as pd


In [2]:
from collections import Counter

def show_table(df, words = [], periods = [], top=50):
    res = pd.DataFrame()
    if words != []:
        if periods != []:
            res = df.loc[words].sort_values(by=periods[0], ascending=False)[periods]
        else:
            res = df.loc[words].sort_values(by=df.columns[0], ascending=False)
    else:
        if periods != []:
            res = df.sort_values(by=periods[0], ascending=False)[periods]
        else:
            res = df.sort_values(by=df.columns[0], ascending=False)
    return res[:top]

def delta(wc1, wc2):
    common = Counter({x:wc1[x]/wc2[x] for x in wc1 if x in wc2 and wc2[x] > 0})
    diff = Counter({x:wc1[x]/0.5 for x in wc1 if wc2[x] == 0})
    return common + diff

def normalize(counter):
    tot = sum(counter.values())
    return Counter(dict({x:counter[x]/tot for x in counter}))

### Variabelen *word_periods* er en container for ord og perioder som undersøkes. 

Alt er aggregert i datasettet, så det må genereres på nytt fra server ved eventuelle endringer


In [91]:
word_periods = {
    'words': {
        'jøde':"jøde Jøde jøden Jøden Jøder jødene Jødene".split(),
        'land':"israelitt israelit Israel Israelit israelitt Israelitt hebreer Hebreer hebræer Hebræer".split(),
        'kristendom': "kristendom kristendommen kirke kirkelig kirkelige kristelig kristelige".split(),
        'religion' : "religion religions religioner religiøs religiøse".split(),
    },

    'periods': {
        '20_tallet' : (1920,1930),
        '30_tallet' : (1930,1940),
    }
}


# Få tak i data

Hent data fra json-filer sendt i mail. Enkleste måte å få tak i den på er å legge den i samme mappe som notebooken. Da slipper du unna sti-håndteringen. Fila **haakon_trigram_data.json** inneholder opptellingene, mens **haakon_trigram_assoc.json** inneholder assosiasjonsmålene.

In [3]:
ord_data = json.load(open('frode_jode.json'))
ord_assoc = json.load(open('frode_jode_assoc.json'))
ord_pmi = json.load(open('frode_jode_pmi.json'))

Dataene gjøres om til en såkalt dataframe for å bruke innebygde metoder. Legger dataene i variabelen **df_ord** og **df_assoc**. Her vil jeg følge en privat konvensjon om å la dataframe-objekter få prefikset df. Objektene starter ut som json-objekter som transformeres til datarammer.

In [4]:
df_ord = pd.DataFrame(ord_data)
df_assoc = pd.DataFrame(ord_pmi)

### Inspiser data

Inspiser dataene med kommandoen show_table(). Den tar tre argument der første argument er en dataframe, andre er de ordene som skal undersøkes, og tredje argument er de kolonnene. Kolonnenavnene er som er definert i **word_periods** ovenfor. For å se alt i datarammen er det bare å la de to siste argumentene stå åpen.

Her er et par eksempler

In [7]:
# vis alt, data sorteres på første kolonne -som du ser er det en del data, men det er muligheter til å forminske.

show_table(df_assoc)

,jøde_20_tallet,jøde_30_tallet,kristendom_20_tallet,kristendom_30_tallet,land_20_tallet,land_30_tallet,religion_20_tallet,religion_30_tallet
Gojim,19.049668,16.900359,NaN,NaN,NaN,NaN,NaN,NaN
greker,15.131889,15.132887,NaN,NaN,NaN,NaN,NaN,NaN
evige,13.006124,10.830866,NaN,NaN,NaN,NaN,NaN,NaN
araberne,12.362319,11.809212,NaN,NaN,NaN,NaN,NaN,NaN
Palestina,12.211324,9.617855,NaN,NaN,NaN,NaN,NaN,NaN
polsk,11.680223,7.141991,NaN,NaN,NaN,NaN,NaN,NaN
Ut,6.459329,NaN,NaN,-1.294952,NaN,NaN,NaN,NaN
navn,6.296230,4.603625,NaN,NaN,NaN,NaN,NaN,NaN
hjem,6.026567,6.006235,1.181232,0.425658,NaN,NaN,NaN,NaN
skaffe,6.023100,3.467976,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Vis for bare enkelte ord, og enkelte kolonner, dataene sorteres på den første kolonnen i perioden . 

show_table(df_assoc, 
           words = 'Gojim gojim'.split(), 
           periods = 'jøde_20_tallet jøde_30_tallet'.split())

,jøde_20_tallet,jøde_30_tallet
Gojim,19.049668,16.900359
gojim,NaN,NaN


## Data kan inspiseres hver for seg

Om du ønsker å se de viktigste ordene for en gitt periode er det bar å la ord-argumentet stå åpent.

In [17]:
show_table(df_assoc, 
           periods= ['kristendom_30_tallet'])

,kristendom_30_tallet
Lundrlng,10.457690
Ilegvik,10.429493
Skago,10.414386
Rancm,10.315322
Prestenes,10.290035
Katb,10.265994
BJølsen-,10.236848
Ilavstein,10.166459
Konfirmasjonsgudstjeneste,10.166459
Kath,10.044203


### show_table() 

har også et parameter *top* som brukes til å regulere størrelsen. Den har default 50

In [18]:
# se på alle kolonnene, sortert etter de viktigste ordene på 20-tallet

show_table(df_assoc, top=3)

,jøde_20_tallet,jøde_30_tallet,kristendom_20_tallet,kristendom_30_tallet,land_20_tallet,land_30_tallet,religion_20_tallet,religion_30_tallet
Gojim,19.049668,16.900359,NaN,NaN,NaN,NaN,NaN,NaN
greker,15.131889,15.132887,NaN,NaN,NaN,NaN,NaN,NaN
evige,13.006124,10.830866,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# 

show_table(df_assoc, 
          periods = ['religion_20_tallet', 'religion_30_tallet'],
          top = 10)

,religion_20_tallet,religion_30_tallet
folketoner,12.450441,11.967127
grunntone,12.259142,8.818690
evangelisklutherske,12.238081,12.041913
Psykoanalyse,11.702749,9.816200
grublerier,11.421792,10.265469
etiske,11.282002,10.398522
estetikk,11.238081,NaN
kirkelige,11.185563,11.236734
evangelisk-lutherske,11.176680,10.456950
arbeidsliv,10.510138,11.405356


In [108]:
# se på kristendommen

show_table(df_assoc, 
          periods = ['religion_20_tallet', 'religion_30_tallet'],
          top = 10)


,religion_20_tallet,religion_30_tallet
folketoner,12.450441,11.967127
grunntone,12.259142,8.818690
evangelisklutherske,12.238081,12.041913
Psykoanalyse,11.702749,9.816200
grublerier,11.421792,10.265469
etiske,11.282002,10.398522
estetikk,11.238081,NaN
kirkelige,11.185563,11.236734
evangelisk-lutherske,11.176680,10.456950
arbeidsliv,10.510138,11.405356
